<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [124]:
import pickle

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [82]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv("../../data/train_sessions.csv", index_col="session_id")
test_df = pd.read_csv("../../data/test_sessions.csv", index_col="session_id")

# приведем колонки time1, ..., time10 к временному формату
times = ["time%s" % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by="time1")

# посмотрим на заголовок обучающей выборки
train_df.head(2)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0


In [83]:
train_df.sort_values(by='session_id').head(2)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [84]:
train_df.head(2)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0


In [85]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ["site%s" % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype("int")
test_df[sites] = test_df[sites].fillna(0).astype("int")

# загрузим словарик сайтов
with open(r"../../data/site_dic_topic4.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(
    list(site_dict.keys()), index=list(site_dict.values()), columns=["site"]
)
print(u"всего сайтов:", sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [86]:
# наша целевая переменная
y_train = train_df["target"]

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop("target", axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [104]:
print(full_df.shape)
print(idx_split)

(336358, 20)
253561


In [105]:
# small
train_df[sites].fillna(0).to_csv('train_sessions_text.txt', 
                                 sep=' ', index=None, header=None)
test_df[sites].fillna(0).to_csv('test_sessions_text.txt', 
                                sep=' ', index=None, header=None)

In [106]:
!head -3 train_sessions_text.txt

"head" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


ДОЛЖНО ВЫГЯДЕТЬ ТАК  
56 55 0 0 0 0 0 0 0 0  
56 55 56 55 0 0 0 0 0 0  
946 946 951 946 946 945 948 784 949 946

In [125]:
%%time
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)
with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)
print(X_train.shape, X_test.shape)

(253561, 50000) (82797, 50000)
CPU times: total: 4.16 s
Wall time: 7.43 s


Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [108]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head(3)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946


In [109]:
len(pd.Series(full_sites.values.ravel()).unique())

48372

Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [110]:
site_dict.keys()

dict_keys(['www.abmecatronique.com', 'groups.live.com', 'majeureliguefootball.wordpress.com', 'cdt46.media.tourinsoft.eu', 'www.hdwallpapers.eu', 'img378.imageshack.us', 'ecologie.nature.free.fr', 'www.ibcn.intec.ugent.be', 'kissanime.com', 'www.carolineconduiteformation.com', 'images.mystockphoto.com', 'journalph.csphares.qc.ca', 'www.uqo.ca', 'd8d94e0wul1nb.cloudfront.net', 'openapi.elong.com', 'flamenco-o.blogspot.com', 'www.pages-annuaire.net', 'smart2000.pagesperso-orange.fr', 'fast.forbes.com', 'i1-js-14-3-01-10077-536503633-i.init.cedexis-radar.net', 'i1-js-14-3-01-11074-716595896-i.init.cedexis-radar.net', 'www.pacajob.com', 'mathaa.epfl.ch', 'cbv.sfr.bench.cedexis.com', 'fbcdn-sphotos-b-a.akamaihd.net', 'www.mystere-tv.com', 'www.mon-ip.fr', 'www.aqua-passion.com', 'reunion.la1ere.fr', 'marketing.kognitio.com', 'www.calvin-thomas.com', 'i1-js-14-3-01-11074-157631352-i.init.cedexis-radar.net', 'download.allin1convert.com', 'i1-js-14-3-01-11074-278531719-i.init.cedexis-radar.net

In [111]:
from scipy.sparse import csr_matrix

In [112]:
# csr_matrix?

In [113]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(
    (
        [1] * sites_flatten.shape[0],
        sites_flatten,
        range(0, sites_flatten.shape[0] + 10, 10),
    )
)[:, 1:]

In [114]:
full_sites.tail(2)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
82796,5828,23,21,804,21,3350,23,894,21,961
82797,21,1098,1098,1098,1098,1098,1098,1098,1098,1098


In [115]:
sites_flatten

array([  56,   55,    0, ..., 1098, 1098, 1098])

In [116]:
full_sites_sparse

<336358x48371 sparse matrix of type '<class 'numpy.int32'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [205]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

<font color='red'>КАК БУДЕТ ВЫГЛЯДЕТЬ КОД ДЛЯ СИТУАЦИИ КОГДА ИСПОЛЬЗУЕТСЯ sites_flatten = full_sites.values.flatten()  full_sites_sparse = csr_matrix(</font>

In [121]:
def get_auc_lr_valid(X, y, C=1.0, ratio=0.9, seed=17):
    """
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state логистической регрессии
    """
    index_to_divide = int(round(ratio * X.shape[0]))
    logit = LogisticRegression(C=C, n_jobs=-1, random_state=seed, solver='lbfgs') # For small datasets, ‘liblinear’ is a good choice
    lr = logit.fit(X[:index_to_divide, :], y[:index_to_divide])
    y_pred = lr.predict_proba(X[index_to_divide:, :])[:, 1]
    score = roc_auc_score(y[index_to_divide:], y_pred)
    print(round(logit.score(X, y), 3))

In [145]:
def adm223_get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    train_len = int(round(ratio * X.shape[0]))
    X_train = X[:train_len, :]
    X_valid = X[train_len:, :]
    y_train = y[:train_len]
    y_valid = y[train_len:]
    
    logit = LogisticRegression(C=C, random_state=seed, n_jobs=-1)
    
    logit.fit(X_train,y_train)
    
    valid_pred = logit.predict_proba(X_valid)[:, 1]
    
    return roc_auc_score(y_valid, valid_pred)

In [136]:
adm223_get_auc_lr_valid(X_train, y_train)

0.9146685651348492

In [126]:
def get_auc_lr_valid(X, y, C=1.0, seed=17, ratio = 0.9):
    # Split the data into the training and validation sets
    idx = int(round(X.shape[0] * ratio))
    # Classifier training
    lr = LogisticRegression(C=C, random_state=seed, solver='lbfgs', max_iter=500).fit(X[:idx, :], y[:idx])
    # Prediction for validation set
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # Calculate the quality
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [257]:
get_auc_lr_valid(X_train, y_train)

0.9146685651348492

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [128]:
# функция для записи прогнозов в файл
def write_to_submission_file(
    predicted_labels, out_file, target="target", index_label="session_id"
):
    predicted_df = pd.DataFrame(
        predicted_labels,
        index=np.arange(1, predicted_labels.shape[0] + 1),
        columns=[target],
    )
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [129]:
lr = LogisticRegression(C=1.0, random_state=17, solver='lbfgs', max_iter=500).fit(X_train, y_train)

# Make a prediction for test data set
y_test = lr.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted
write_to_submission_file(y_test, 'baseline_12.csv')

In [ ]:
session_start_hour = train_times['time1'].apply(lambda ts: 100 * ts.hour + int(ts.minute / 10)).values

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [148]:
train_df.head(2)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0


In [173]:
copy_df.index[0]

21669

In [150]:
copy_df = train_df

In [180]:
str(copy_df[copy_df.index == 21669]["time1"].dt.date)

'session_id\n21669    2013-01-12\nName: time1, dtype: object'

In [167]:
# def add_YYYYMM_features(df):
#     a = df["time1"].dt.to_period("Y")[0]
#     b = df["time1"].dt.to_period("M")[1]
#     df["time1_improved"] = str(a)
    # df[f"years_months{}"] = 1

In [ ]:
# add_YYYYMM_features(copy_df)
# copy_df.tail(2).T

In [184]:
def add_YYYYMM_features(df, X_sparse):
    year = df['time1'].apply(lambda ts: ts.year)
    month = df['time1'].apply(lambda ts: ts.month)
    X = hstack([X_sparse, year.values.reshape(-1, 1), 
                month.values.reshape(-1, 1)])
    return X

Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [195]:
X_train_new = add_YYYYMM_features(train_df.fillna(0), X_train)
X_test_new = add_YYYYMM_features(test_df.fillna(0), X_test)

In [187]:
get_auc_lr_valid(X_train, y_train)

0.9146685651348492

ЕЩЁ СПОСОБ

In [192]:
new_feat_train = pd.DataFrame(index=train_df.index)
new_feat_test = pd.DataFrame(index=test_df.index)

In [194]:
new_feat_train['year_month'] = train_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
new_feat_test['year_month'] = test_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
new_feat_test['year_month'].head()

session_id
1    201410
2    201407
3    201412
4    201411
5    201405
Name: year_month, dtype: int64

In [202]:
scaler = StandardScaler()
scaler.fit(new_feat_train['year_month'].values.reshape(-1,1))

new_feat_train['year_month_scaler'] = scaler.transform(new_feat_train['year_month'].values.reshape(-1,1))
new_feat_test['year_month_scaler'] = scaler.transform(new_feat_test['year_month'].values.reshape(-1,1))
new_feat_train.tail(3)

,year_month,year_month_scaler
session_id,,
12221,201404,0.681626
156968,201404,0.681626
204762,201404,0.681626


In [206]:
X_train_sparse.shape, new_feat_train['year_month_scaler'].values.reshape(-1,1).shape

((253561, 48371), (253561, 1))

In [207]:
%%time
X_train_sparse_new = csr_matrix(hstack([X_train_sparse, new_feat_train['year_month_scaler'].values.reshape(-1,1)]))

CPU times: total: 46.9 ms
Wall time: 78.6 ms


In [208]:
%%time
X_test_sparse_new = csr_matrix(hstack([X_test_sparse, new_feat_test['year_month_scaler'].values.reshape(-1,1)]))

CPU times: total: 15.6 ms
Wall time: 22 ms


In [209]:
X_train_sparse.shape, X_train_sparse_new.shape

((253561, 48371), (253561, 48372))

In [210]:
X_train_sparse_new.shape, y_train.shape

((253561, 48372), (253561,))

In [211]:
%%time
get_auc_lr_valid(X_train_sparse_new, y_train)

CPU times: total: 2.62 s
Wall time: 806 ms


0.9197442049621032

In [260]:
std_auc_lr_valid = get_auc_lr_valid(X_train_sparse_new, y_train)
std_auc_lr_valid
# с одним добавленным призаком

0.9197442049621032

In [213]:
print("ROC AUC на выборке", ";", "ROC AUC на выборке с новым отмасштабированным признаком времени(года и месяца)", " Разница")
print(f"{get_auc_lr_valid(X_train, y_train)} - {get_auc_lr_valid(X_train_sparse_new, y_train)} = {get_auc_lr_valid(X_train, y_train) - get_auc_lr_valid(X_train_sparse_new, y_train)}")

Разница во времени:
ROC AUC на выборке ; ROC AUC на выборке с новым отмасштабированным признаком времени(года и месяца)  Разница


0.9146685651348492 - 0.9197442049621032 = -0.005075639827254008


Видим отрицательное число, то есть добавив признак мы улучшили эффектинвость нашей класификации, НО абсолютно не заметно

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [238]:
train_df['time1'].head(1)

session_id
21669   2013-01-12 08:05:57
Name: time1, dtype: datetime64[ns]

In [239]:
new_feat_train['start_hour'] = train_df['time1'].apply(lambda ts: ts.hour)
new_feat_test['start_hour'] = test_df['time1'].apply(lambda ts: ts.hour)
new_feat_test['start_hour'].head(3)

session_id
1    11
2    11
3    15
Name: start_hour, dtype: int64

In [227]:
scaler_hour = StandardScaler()
scaler_hour.fit(new_feat_train['start_hour'].values.reshape(-1,1))

new_feat_train['start_hour_scaler'] = scaler_hour.transform(new_feat_train['start_hour'].values.reshape(-1,1))
new_feat_test['start_hour_scaler'] = scaler_hour.transform(new_feat_test['start_hour'].values.reshape(-1,1))
new_feat_train.tail(3)

,year_month,year_month_scaler,start_hour,start_hour_scaler
session_id,,,,
12221,201404,0.681626,8,-1.243533
156968,201404,0.681626,36,0.374713
204762,201404,0.681626,53,1.357220


In [247]:
new_feat_train['morning'] = new_feat_train['start_hour'].apply(lambda ts: 1 if ts <= 11 else 0)
new_feat_test['morning'] = new_feat_test['start_hour'].apply(lambda ts: 1 if ts <= 11 else 0)
new_feat_train.sort_values(by="start_hour", ascending=False).tail(3)

,year_month,year_month_scaler,start_hour,start_hour_scaler,morning
session_id,,,,,
70133,201311,-1.508868,7,-0.318821,1
173788,201311,-1.508868,7,-0.838972,1
225850,201401,0.610965,7,-1.070150,1


In [ ]:
# new_feat_train['morning_scaler'] = scaler_hour.transform(new_feat_train['start_hour'].values.reshape(-1,1))
# new_feat_test['morning_scaler'] = scaler_hour.transform(new_feat_test['start_hour'].values.reshape(-1,1))
# new_feat_train.tail(3)

In [250]:
new_feat_train['start_month'] = train_df['time1'].apply(lambda ts: ts.month)
new_feat_test['start_month'] = test_df['time1'].apply(lambda ts: ts.month)
new_feat_test['start_month'].head(3)

session_id
1    10
2     7
3    12
Name: start_month, dtype: int64

start_month и start_hour  
-  
-  

In [249]:
X_train_sparse_new.shape, new_feat_train['start_month'].values.reshape(-1,1).shape

((253561, 48372), (253561, 1))

In [253]:
%%time
X_train_sparse_start_month_start_hour = csr_matrix(hstack([X_train_sparse_new, new_feat_train['start_hour'].values.reshape(-1,1)]))

CPU times: total: 0 ns
Wall time: 34 ms


In [266]:
%%time
X_test_sparse_start_month_start_hour = csr_matrix(hstack([X_test_sparse_new, new_feat_test['start_hour'].values.reshape(-1,1)]))

CPU times: total: 0 ns
Wall time: 10.2 ms


In [265]:
X_train_sparse_new.shape, X_train_sparse_start_month_start_hour.shape

((253561, 48372), (253561, 48373))

In [264]:
%%time
auc_train_start_month_start_hour = get_auc_lr_valid(X_train_sparse_start_month_start_hour, y_train)
print("Точнсть С одним признаком:", std_auc_lr_valid)
print('Точность увеличилась на: ', auc_train_start_month_start_hour - std_auc_lr_valid, "\n", auc_train_start_month_start_hour, "\n")

С одним признаком: 0.9197442049621032
Точность увеличилась на:  0.036148760320648954 
 0.9558427260408968 

CPU times: total: 9.48 s
Wall time: 2.76 s


start_month и morning  
-  
-  

In [267]:
%%time
X_train_sparse_start_month_morning = csr_matrix(hstack([X_train_sparse_new, new_feat_train['morning'].values.reshape(-1,1)]))

CPU times: total: 15.6 ms
Wall time: 28.8 ms


In [268]:
%%time
X_test_sparse_start_month_morning = csr_matrix(hstack([X_test_sparse_new, new_feat_test['morning'].values.reshape(-1,1)]))

CPU times: total: 0 ns
Wall time: 9.05 ms


In [269]:
X_train_sparse_new.shape, X_train_sparse_start_month_morning.shape

((253561, 48372), (253561, 48373))

In [275]:
%%time
auc_train_start_month_morning = get_auc_lr_valid(X_train_sparse_start_month_morning, y_train)
print("Точнсть С одним признаком:", std_auc_lr_valid)
print('Точность увеличилась на: ', auc_train_start_month_morning - std_auc_lr_valid, "\n", auc_train_start_month_morning, "\n")
print(f"Разница start_hour c morning: {auc_train_start_month_start_hour - auc_train_start_month_morning}")

Точнсть С одним признаком: 0.9197442049621032
Точность увеличилась на:  0.028443660662096693 
 0.9481878656241999 

Разница с вариантом start_month и start_hour: 0.00765486041669694
CPU times: total: 2.67 s
Wall time: 823 ms


start_month, start_hour и morning   
-  
-  

In [272]:
%%time
X_train_sparse_start_month_start_hour_morning = csr_matrix(hstack([X_train_sparse_start_month_start_hour, new_feat_train['morning'].values.reshape(-1,1)]))

CPU times: total: 15.6 ms
Wall time: 32.5 ms


In [273]:
%%time
X_test_sparse_start_month_start_hour_morning = csr_matrix(hstack([X_test_sparse_start_month_start_hour, new_feat_test['morning'].values.reshape(-1,1)]))

CPU times: total: 0 ns
Wall time: 10.8 ms


In [274]:
X_train_sparse_new.shape, X_train_sparse_start_month_start_hour_morning.shape

((253561, 48372), (253561, 48374))

In [278]:
%%time
auc_train_start_month_start_hour_morning = get_auc_lr_valid(X_train_sparse_start_month_start_hour_morning, y_train)
print("Точнсть С одним признаком:", std_auc_lr_valid)
print('Точность увеличилась на: ', auc_train_start_month_start_hour_morning - std_auc_lr_valid, "\n", auc_train_start_month_start_hour_morning, "\n")
print(f"Разница последнйи вариант(все три) и start_hour: {auc_train_start_month_start_hour_morning - auc_train_start_month_start_hour}")

Точнсть С одним признаком: 0.9197442049621032
Точность увеличилась на:  0.03844835852547468 
 0.9581925634875779 

Разница последнйи вариант(все три) и start_hour: 0.0023498374466810468
CPU times: total: 10.2 s
Wall time: 2.98 s


### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [285]:
get_auc_lr_valid(X_train_sparse_start_month_start_hour_morning, y_train)

0.9581925634875779

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [292]:
%%time
max_AUC = 0
max_C = 0
for i in np.logspace(-3, 1, 10):
    current = get_auc_lr_valid(X_train_sparse_start_month_start_hour_morning, y_train, i)
    if current > max_AUC:
        max_AUC = current
        max_C = i

CPU times: total: 1min 10s
Wall time: 21.8 s


In [293]:
max_AUC, max_C

(0.9618320489688853, 0.1668100537200059)

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [295]:
logit_Best = LogisticRegression(random_state=17, n_jobs=-1, C=max_C)
logit_Best.fit(X_train_sparse_start_month_start_hour_morning, y_train)
test_all_features_BEST_C = logit_Best.predict_proba(X_test_sparse_start_month_start_hour_morning)[:,1]
test_all_features_BEST_C

array([3.89944564e-04, 2.77012920e-08, 4.46721532e-07, ...,
       9.87379395e-04, 8.18584564e-05, 8.48993184e-06])

In [ ]:
write_to_submission_file(test_all_features_BEST_C, 'submit_2.csv')
